This notebook is intended to visualize the final tracks generated while running WALDO.

not sure if useful.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys
import pickle
import pathlib

from mpltools import style
from mpltools import layout

sys.path.append('..'); import pathcustomize, about
about.about()

import pathcustomize

import waldo.wio as wio
import waldo.tape as tape
import waldo.collider as collider
from waldo.viz.network.notebook import look

style.use('ggplot')

Python 2.7.9 (default, Apr 14 2015 12:54:25) [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux2, Host: cody


store location of all blobs files
and map blob id to file using dict.

In [7]:
def blobs_file_reader(bf):
    with open(bf, 'r') as f:
        lines = f.readlines()
    #print lines[0]
    lines = lines[1:]
    dlines = []
    for l in lines:
        parts = l.strip().split()
        dlines.append([float(p) for (i, p) in enumerate(parts) if i < 10])
    df = pd.DataFrame(dlines)
    df.rename(columns={0:'frame', 1:'t', 2:'x', 3:'y'}, inplace=True)
    return df        
        

get a graph of 

In [8]:
def show_blob_stuff(df):
    #print len(df), 'long'
    #print df.head()

    fig, ax = plt.subplots()
    ax.plot(df['x'], df['y'], '.')
    ax.plot(df['x'], df['y'], alpha=0.2)
    ax.set_ylabel('y')
    ax.set_xlabel('x')

    fig, ax = plt.subplots(2,1)

    ax[0].plot(df['t'], df['x'], alpha=0.2)
    ax[0].plot(df['t'], df['x'], '.')
    ax[0].set_ylabel('x')

    ax[1].plot(df['t'], df['y'], alpha=0.2)
    ax[1].plot(df['t'], df['y'], '.')
    ax[1].set_ylabel('y')
    ax[1].set_xlabel('t')
  
    fig, ax = plt.subplots(2,1)

    ax[0].plot(df['frame'], df['x'], alpha=0.2)
    ax[0].plot(df['frame'], df['x'], '.')
    ax[0].set_ylabel('x')

    ax[1].plot(df['frame'], df['y'], alpha=0.2)
    ax[1].plot(df['frame'], df['y'], '.')
    ax[1].set_ylabel('y')
    ax[1].set_xlabel('frame')    
    
    plt.show()

In [10]:
ex_id = '20130318_131111' # lots of gaps
#ex_id = '20130614_120518' # lots of false positives
#ex_id = '20130702_135704' # testset

fullpath = pathlib.Path('/home/projects/worm_movement/Data/waldo_paper_review/WALDO/') / ex_id / 'blobs_files'

experiment = wio.Experiment(experiment_id=ex_id, fullpath=fullpath)
graph = experiment.graph.copy()


DATA_DIR = './../../data/chore/{eid}/'.format(eid=ex_id)

blobs_files = glob.glob(DATA_DIR + '*.blobs')
for i in blobs_files[:1]:
    print(i)
    
id_map = {}
for bf in blobs_files:
    with open(bf, 'r') as f:
        l = f.readline()
        bid = int(l.strip().split('%')[-1])
        id_map[bid] = bf

MWTSummaryError: Could not find summary file in target path: /home/projects/worm_movement/Data/waldo_paper_review/WALDO/20130318_131111/blobs_files

In [ ]:
graph2 = pickle.load(open('{d}/graph.pickle'.format(d=DATA_DIR)))
#graph2 = pickle.load(open('./../../data/chore/{eid}/graph.pickle'.format(d=DATA_DIR, eid=ex_id)))
#graph2._gap_blobs

### create index of node counts

In [ ]:
node_gap_count = {}
gap_index = {}
for n in graph2.nodes():
    if n == 90:
        print 'what?'
    node_gap_count[n] = 0
    #gap_index[n] = []
    
#print gap_index
print set(gap_index.values())
non_consensus = []
con = 0
for b1, b2 in graph2._gap_blobs:
    n1 = graph2.where_is(b1)
    n2 = graph2.where_is(b2)
    if n1 == n2:
        con += 1
        node_gap_count[n1] = 1 + node_gap_count[n1]
        
        node_gaps = gap_index.get(n1, [])
        if node_gaps is None:
            print 'none problem happended again'
            print n1, n2, b1, b2
            node_gaps = []
        node_gaps.append([b1, b2])
        gap_index[n1] = node_gaps
    else:
        #print 'not collapsed', b1, b2, n1, n2
        non_consensus.append([b1, b2, n1, n2])
        
print con, 'consensus'
print len(non_consensus), 'non consensus'

In [ ]:
gap_counts = node_gap_count.values()
nodes = node_gap_count.keys()

n_bins = max(gap_counts)
gc = [c for c in gap_counts if c > 0]
print len(nodes), 'total nodes'
print len(gc), 'nodes with gaps'
fig, ax = plt.subplots()

ax.hist(gap_counts, bins=n_bins)
plt.ylabel('count')
plt.xlabel('number of gaps bridged')
plt.title('all nodes')

fig, ax = plt.subplots()

plt.hist(gc, bins=n_bins)
plt.ylabel('count')
plt.xlabel('number of gaps bridged')
plt.title('nodes with bridged gaps')
plt.show()

gap_counts, nodes = zip(*[(c, n) for (c, n) in sorted(zip(gap_counts, nodes), reverse=True)])

In [ ]:
nodes[:5]

In [ ]:
node = nodes[0]
b_df = collider.consolidate_node_data(graph2, experiment, node)
df = b_df[['time', 'blob']]
x, y = zip(*b_df['centroid'])
df['x'] = x
df['y'] = y
df.sort(inplace=True)
ordered_components = [int(b) for b in df['blob'].drop_duplicates()]

fig, ax = plt.subplots()
for bid in ordered_components:
    b = df[df['blob'] == bid]
    x, y = b['x'], b['y']
    ax.plot(x, y)

plt.show()

In [ ]:
def format_gap_data(gaps, ordered_components, df):
    ordered = 0
    unordered = 0
    gap_locations = []
    gap_lines = []
    sx = []
    sy = []
    st = []
    et = []
    ex = []
    ey = []

    gdict = {}
    for g1, g2 in gaps:
        if g1 not in ordered_components:
            print g1, 'gap start not found in components'
            continue
        if g2 not in ordered_components:
            print g2, 'gap end not found in components'
            continue
            
        i1, i2 = ordered_components.index(g1), ordered_components.index(g2)
        is_ordered = False
        for i in range(6):
            if i1 + i == i2:
                #print i, 'spaces between unordered'
                gdict[g1] = g2
                is_ordered = True
        if not is_ordered:
            unordered += 1
    print unordered, 'unordered'


    for g1 in ordered_components:
        if g1 in gdict:
            g2 = gdict[g1]
            b1 = df[df['blob'] == g1].iloc[-1]
            t1, x1, y1 = b1['time'], b1['x'], b1['y']

            b2 = df[df['blob'] == g2].iloc[0]
            t2, x2, y2 = b2['time'], b2['x'], b2['y']
            gx = x1 #float(x1 + x2) / 2.0
            gy = y1 #float(y1 + y2) / 2.0

            st.append(t1)
            sx.append(x1)
            sy.append(y1)   

            et.append(t2)
            ex.append(x2)
            ey.append(y2)       
    mid_x = []
    mid_y = []
    for x1, y1, x2, y2 in zip(sx, sy, ex, ey):
        #ax.plot([x1, x2], [y1, y2], color='b', alpha=0.5, lw=1)
        mid_x.append((x1 + x2) / 2.0)
        mid_y.append((y1 + y2) / 2.0)
    return st, sx, sy, et, ex, ey, mid_x, mid_y


def show_gap_history(graph, experiment, gap_index, node):
    
    b_df = collider.consolidate_node_data(graph2, experiment, node)
    df = b_df[['time', 'blob']]
    x, y = zip(*b_df['centroid'])
    df['x'] = x
    df['y'] = y
    df.sort(inplace=True)
    ordered_components = [int(b) for b in df['blob'].drop_duplicates()]
    gaps = gap_index[node]
    print len(gaps), 'gaps'
    st, sx, sy, et, ex, ey, mid_x, mid_y = format_gap_data(gaps, ordered_components, df)

    fig, ax = plt.subplots()
    fig.set_size_inches(18.5,10.5)
    for bid in ordered_components:
        b = df[df['blob'] == bid]
        x, y = b['x'], b['y']
        ax.plot(x, y, lw=4)

    ax.plot(mid_x, mid_y, 'x', lw=0, color='black', label='missing data', markersize=20)
    ax.legend()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    
show_gap_history(graph=graph2, experiment=experiment, gap_index=gap_index, node=nodes[0])
plt.show()

In [ ]:
show_gap_history(graph=graph2, experiment=experiment, gap_index=gap_index, node=nodes[1])
plt.show()

In [ ]:
show_gap_history(graph=graph2, experiment=experiment, gap_index=gap_index, node=nodes[2])
plt.show()

In [ ]:
show_gap_history(graph=graph2, experiment=experiment, gap_index=gap_index, node=nodes[50])
plt.show()

In [ ]:
bf = id_map[4]
print bf
df = blobs_file_reader(bf)
show_blob_stuff(df)